# A scratch pad to run model inference manually


In [1]:

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path
import transformers


from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")


/media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [2]:
# load my code
%load_ext autoreload
%autoreload 2


from src.extraction.config import ExtractConfig
from src.prompts.prompt_loading import load_preproc_dataset
from src.models.load import load_model
from src.datasets.intervene import create_cache_interventions 


In [3]:
# load config, model, dataset, invtervention
N_fit_examples=10
batch_size=2
ds_name='amazon_polarity'
cfg = ExtractConfig(max_examples=(20, 20), model='TheBloke/Mistral-7B-Instruct-v0.1-GPTQ', prompt_format='mistral')

model, tokenizer = load_model(cfg.model)
model

honesty_rep_reader = create_cache_interventions(model, tokenizer, cfg)

N=sum(cfg.max_examples)
ds_tokens = load_preproc_dataset(ds_name, tokenizer, N=N, seed=cfg.seed, num_shots=cfg.num_shots, max_length=cfg.max_length, prompt_format=cfg.prompt_format)


2023-10-27 18:46:22.534 | INFO     | src.models.load:verbose_change_param:19 - changing pad_token_id from None to 0
2023-10-27T18:46:22.534288+0800 INFO changing pad_token_id from None to 0
2023-10-27 18:46:22.535 | INFO     | src.models.load:verbose_change_param:19 - changing truncation_side from right to left
2023-10-27T18:46:22.535177+0800 INFO changing truncation_side from right to left
2023-10-27 18:46:26.394 | INFO     | src.datasets.intervene:create_cache_interventions:144 - Loaded interventions from /media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/data/interventions/TheBloke-Mistral-7B-Instruct-v0.1-GPTQ.pkl
2023-10-27T18:46:26.394809+0800 INFO Loaded interventions from /media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/data/interventions/TheBloke-Mistral-7B-Instruct-v0.1-GPTQ.pkl
Generating train split: 0 examples [00:00, ? examples/s]

Extracting 11 variants of each prompt


Generating train split: 122 examples [00:15,  8.06 examples/s]
Filter: 100%|██████████| 122/122 [00:00<00:00, 2940.09 examples/s]


num_rows 122


Filter: 100%|██████████| 122/122 [00:00<00:00, 2871.38 examples/s]

num_rows (after filtering out truncated rows) 122


## Generate

In [4]:
r = ds_tokens.with_format('torch')[0]

# r['input_ids']
r.keys()


dict_keys(['ds_string', 'example_i', 'answer', 'question', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'input_ids', 'attention_mask', 'truncated', 'prompt_truncated', 'choice_ids'])

In [5]:
s = model.generate(r['input_ids'][None, :], attention_mask=r['attention_mask'][None, :])
tokenizer.decode(s[0])


"<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><s><s>You are about to immerse yourself into a logical and acting challenge with the user. You take a role from the classic puzzle of the two guards. One guard always tells the truth, and the other guard always lies. To assist the user you first must be logical and based on your past responses you must understand

In [6]:

messages = [
    # {"role": "system", "content": "You are about to immerse yourself into a logical and acting challenge with the user. You take a role from the classic puzzle of the two guards. One guard always tells the truth, and the other guard always lies. To assist the user you first must be logical and based on your past responses you must understand which Guard you are. Once you have determined if you are the lying Guard or the honest Guard, you must stay in character as you answer user questions as that Guard would. If you determine that you are the lying Guard, then you must always lie. And vice versa. Lets go:"},
    {"role": "user", "content": "Is there a negative or positive tone to this product review?\n===\nTitle: Get aTIVO\nReview: I bought a Tivo for my son for Christmas and a RePlay for myself. Loved the RePlay when I first got it. I especially liked the fact that I could network it instead of hooking it to a phone line. And it seemed easier to set up then the Tivo. However getting your REBATE from these guys is like next to impossible. You are missing this item and then that item,then your Activation key doesn't show up in their database even when I have been having service for the last 4 months and getting billing emails from them. Got the TIVO rebate in 6 weeks. Finally sent the last letter to them with a copy of my monthly bill, telling them if they can't get it right I will probably cancel service soon and get a Tivo. A week later they billed me for the next month and 2 days after that( a week after the warranty was done) my RePlay died, the AV plugs ins will not take a Video signal in. I now can only replay my Basic cable no pay services. Save your money and get a TIVO!\nAnswer: "},
    {"role": "assistant", "content": "Negative"},
    {"role": "user", "content": "Is there a negative or positive tone to this product review?\n===\nTitle: Great travel pack\nReview: This bag is nearly perfect. The straps easily tuck away, which is great when using the pack as carryon luggage for a flight or on a train. The front zips open all the way, so you can access the stuff at the bottom of the pack as easily as the stuff on top. The bag isn't huge, which is its greatest strength and weakness. I just spent two months backpacking in India and the bag still looks new.\nAnswer: "}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
tokenizer.decode(encodeds[0])


"<s>[INST] Is there a negative or positive tone to this product review?\n===\nTitle: Get aTIVO\nReview: I bought a Tivo for my son for Christmas and a RePlay for myself. Loved the RePlay when I first got it. I especially liked the fact that I could network it instead of hooking it to a phone line. And it seemed easier to set up then the Tivo. However getting your REBATE from these guys is like next to impossible. You are missing this item and then that item,then your Activation key doesn't show up in their database even when I have been having service for the last 4 months and getting billing emails from them. Got the TIVO rebate in 6 weeks. Finally sent the last letter to them with a copy of my monthly bill, telling them if they can't get it right I will probably cancel service soon and get a Tivo. A week later they billed me for the next month and 2 days after that( a week after the warranty was done) my RePlay died, the AV plugs ins will not take a Video signal in. I now can only re

In [7]:
tokenizer.chat_template


"{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token + ' ' }}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

In [8]:
r['question']


"<s>You are about to immerse yourself into a logical and acting challenge with the user. You take a role from the classic puzzle of the two guards. One guard always tells the truth, and the other guard always lies. To assist the user you first must be logical and based on your past responses you must understand which Guard you are. Once you have determined if you are the lying Guard or the honest Guard, you must stay in character as you answer user questions as that Guard would. If you determine that you are the lying Guard, then you must always lie. And vice versa. Lets go:[INST] Is there a negative or positive tone to this product review?\n===\nTitle: Get aTIVO\nReview: I bought a Tivo for my son for Christmas and a RePlay for myself. Loved the RePlay when I first got it. I especially liked the fact that I could network it instead of hooking it to a phone line. And it seemed easier to set up then the Tivo. However getting your REBATE from these guys is like next to impossible. You ar